# Flow-Shop Problem-Solving with Google `FunSearch`

> Authors: CUI Guangyuan, LI Songyan, XU Zhuojun

This notebook is the main entrance of our work on the flow-shop problems' solving using Google `FunSearch` via various attempts. Mainly, the applications of existing methods on the problem-solving can be divided into three categories: baseline experiments including the applications of Google `FunSearch` and `OR-Tools` as well as the existing heuristics (only NEH algorithm). New approaches are also proposed in this research, including:

- Trials on different kinds of prompts (Prompt Engineering)
- FunSearch with Curriculum Learning

> Environment setup before the execution (**IMPORTANT**):
> - Our experiments are operated under the anaconda virtual environment
>
> You can follow the instructions below to setup the correct environment:
> - `conda create -n funsearch_env -f environment.yml`

## Dataset for Evaluation and Testing

In [1]:
instance1 = "data/carlier/carlier1.txt"
instance2 = "data/carlier/carlier2.txt"
instance3 = "data/heller/heller1.txt"
instance4 = "data/heller/heller2.txt"
instance5 = "data/reeves/reeves1.txt"
instance6 = "data/reeves/reeves2.txt"
instance7 = "data/reeves/reeves3.txt"

instances = [
    instance1,
    instance2,
    instance3,
    instance4,
    instance5,
    instance6,
    instance7
]

## Baseline Experiments

### Existing Heuristics

In [2]:
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

Parse the input data:

In [3]:
def parse_input(input_data):
    lines = input_data.strip().split('\n')
    n_jobs, n_machines = map(int, lines[0].split())
    
    jobs = []
    for i in range(1, n_jobs + 1):
        job_data = lines[i].split()
        job = []
        for j in range(0, 2 * n_machines, 2):
            machine = int(job_data[j])
            processing_time = int(job_data[j + 1])
            job.append((machine, processing_time))
        jobs.append(job)
    
    return n_jobs, n_machines, jobs

Calculate makespan for a given job sequence:

In [4]:
def calculate_makespan(jobs, job_sequence, n_machines):
    machine_times = [0] * n_machines
    
    for job_idx in job_sequence:
        job = jobs[job_idx]
        for (machine, processing_time) in job:
            if machine == 0:
                machine_times[machine] = machine_times[machine] + processing_time
            else:
                machine_times[machine] = max(machine_times[machine], machine_times[machine - 1]) + processing_time
    
    return machine_times[-1]

Function to visualize the schedule:

In [5]:
def visualize_schedule(jobs, job_sequence, n_machines, algorithm_name):
    # Calculate start and end times for each operation
    machine_times = [0] * n_machines
    schedule = []
    
    for job_idx in job_sequence:
        job = jobs[job_idx]
        job_schedule = []
        
        for machine, processing_time in job:
            if machine == 0:
                start_time = machine_times[machine]
            else:
                start_time = max(machine_times[machine], machine_times[machine - 1])
            
            end_time = start_time + processing_time
            machine_times[machine] = end_time
            job_schedule.append((machine, start_time, end_time))
        
        schedule.append(job_schedule)
    
    # Create the visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Define colors for jobs
    colors = plt.cm.tab10.colors
    
    # Plot each operation
    for i, job_schedule in enumerate(schedule):
        job_idx = job_sequence[i]
        for machine, start, end in job_schedule:
            ax.barh(machine, end - start, left=start, height=0.8, 
                   color=colors[job_idx % len(colors)], alpha=0.8,
                   edgecolor='black', linewidth=1)
            
            # Add job number label
            if end - start > 30:  # Only add text if bar is wide enough
                ax.text(start + (end - start) / 2, machine, f'J{job_idx}', 
                       ha='center', va='center', color='black', fontweight='bold')
    
    # Add legend
    legend_elements = [Patch(facecolor=colors[i % len(colors)], edgecolor='black', label=f'Job {i}')
                      for i in range(len(jobs))]
    ax.legend(handles=legend_elements, loc='upper right')
    
    # Set labels and title
    ax.set_xlabel('Time')
    ax.set_ylabel('Machine')
    ax.set_yticks(range(n_machines))
    ax.set_yticklabels([f'Machine {i}' for i in range(n_machines)])
    ax.set_title(f'Flow Shop Schedule - {algorithm_name}\nMakespan: {machine_times[-1]}')
    
    # Add grid
    ax.grid(True, axis='x', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    return fig

Definition of NEH algorithm:

In [6]:
def neh_algorithm(jobs, n_jobs, n_machines):
    # Calculate total processing time for each job
    job_times = []
    for i, job in enumerate(jobs):
        total_time = sum(time for _, time in job)
        job_times.append((i, total_time))
    
    # Sort jobs by total processing time (descending)
    job_times.sort(key=lambda x: x[1], reverse=True)
    
    # Build sequence incrementally
    sequence = [job_times[0][0]]
    
    for i in range(1, n_jobs):
        job_idx = job_times[i][0]
        best_makespan = float('inf')
        best_position = 0
        
        # Try inserting the job at each possible position
        for j in range(len(sequence) + 1):
            test_sequence = sequence.copy()
            test_sequence.insert(j, job_idx)
            makespan = calculate_makespan(jobs, test_sequence, n_machines)
            
            if makespan < best_makespan:
                best_makespan = makespan
                best_position = j
        
        sequence.insert(best_position, job_idx)
    
    makespan = calculate_makespan(jobs, sequence, n_machines)
    return sequence, makespan

Run and evaluate original NEH algorithm as the baseline:

In [7]:
for ins in instances:
    with open(ins, 'r') as f:
        input_data = f.read()

    n_jobs, n_machines, jobs = parse_input(input_data)

    print(f"Problem: {ins} | Jobs: {n_jobs} | Machines: {n_machines}")

    print(f"Running NEH...")
    start_time = time.time()
    sequence, makespan = neh_algorithm(jobs, n_jobs, n_machines)
    end_time = time.time()

    execution_time = end_time - start_time

    print(f"  Sequence: {sequence}")
    print(f"  Makespan: {makespan}")
    print(f"  Time: {execution_time:.6f} seconds\n")

    # fig = visualize_schedule(jobs, sequence, n_machines, 'NEH')
    # plt.show()

Problem: data/carlier/carlier1.txt | Jobs: 11 | Machines: 5
Running NEH...
  Sequence: [7, 0, 4, 8, 2, 10, 3, 6, 5, 1, 9]
  Makespan: 7038
  Time: 0.000521 seconds

Problem: data/carlier/carlier2.txt | Jobs: 13 | Machines: 4
Running NEH...
  Sequence: [6, 10, 2, 12, 3, 4, 0, 1, 8, 7, 11, 5, 9]
  Makespan: 7376
  Time: 0.000545 seconds

Problem: data/heller/heller1.txt | Jobs: 100 | Machines: 10
Running NEH...
  Sequence: [36, 62, 0, 26, 21, 73, 56, 17, 98, 83, 15, 16, 53, 75, 94, 71, 4, 59, 34, 68, 87, 96, 97, 58, 12, 65, 82, 41, 70, 66, 22, 81, 25, 50, 8, 78, 45, 49, 48, 10, 90, 85, 43, 64, 14, 13, 86, 61, 1, 89, 42, 92, 63, 99, 76, 5, 91, 28, 19, 20, 39, 2, 47, 31, 52, 7, 80, 72, 27, 18, 67, 60, 29, 95, 40, 24, 54, 38, 23, 32, 37, 35, 57, 77, 84, 88, 51, 69, 33, 9, 6, 3, 79, 11, 44, 93, 55, 30, 74, 46]
  Makespan: 519
  Time: 0.372801 seconds

Problem: data/heller/heller2.txt | Jobs: 20 | Machines: 10
Running NEH...
  Sequence: [0, 3, 11, 8, 14, 16, 1, 12, 15, 7, 19, 18, 10, 5, 17, 4

### Google `OR-Tools`

Import necessary libraries:

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import numpy as np

from ortools.sat.python import cp_model

Define the function to read cases from instances in desired formats:

In [ ]:
def read_cases(path):
    cases = []
    with open(path, 'r') as f:
        alldata = f.readlines()
        first_line = alldata[0].split()
        n_jobs, n_machines = int(first_line[0]), int(first_line[1])

        for i in range(1, len(alldata)):
            line = alldata[i]
            jobs_cases = []
            data = line.split()
            for d in range(0, len(data), 2):
                jobs_cases.append((int(data[d]), int(data[d+1])))
            cases.append(jobs_cases)

    return (n_jobs, n_machines), cases

Define the function for plotting Gantt chart of the result:

In [ ]:
def plot_gantt_chart(result_job_schedule, num_jobs, num_machines,
                     title="Flow-Shop Gantt Chart"):
    fig, ax = plt.subplots(figsize=(25, 12))

    colors = list(mcolors.TABLEAU_COLORS.values())
    if num_jobs > len(colors):
        random.seed(4487)
        colors = []
        for _ in range(num_jobs):
            colors.append(f'#{random.randint(0, 0xFFFFFF):06x}')

    for job_id, machine_id, stime, etime in result_job_schedule:
        duration = etime - stime
        rect = patches.Rectangle(
            (stime, num_machines - machine_id - 1),
            duration,
            0.8,
            linewidth=1,
            edgecolor='black',
            facecolor=colors[job_id],
            alpha=0.6,
            label=f'Job-{job_id}' if machine_id == 0 else ''
        )
        ax.add_patch(rect)

        rx, ry = rect.get_xy()
        ax.text(
            rx + duration / 2,
            ry + 0.4,
            f'J{job_id}',
            ha='center',
            va='center',
            color='black',
            fontweight='light'
        )

    ax.set_xlim(0, max([endtime for _, _, _, endtime in result_job_schedule]) + 1)
    ax.set_ylim(0, num_machines)

    ax.set_yticks(np.arange(num_machines) + 0.4)
    ax.set_yticklabels([f'Machine {num_machines - i - 1}' for i in range(num_machines)])

    ax.grid(True, axis='x', linestyle='--', alpha=0.5)

    handles = [patches.Patch(color=colors[i], label=f'Job {i}') for i in range(num_jobs)]
    ax.legend(handles=handles, loc='upper right', ncol=min(5, num_jobs))

    ax.set_title(title)
    ax.set_xlabel('Time')
    ax.set_ylabel('Machines')

    plt.tight_layout()
    plt.show()

Define the main process of problem solving:

In [ ]:
def solve_flowshop(num_jobs, num_machines, jobs_data):

    model = cp_model.CpModel()

    """Create interval variables"""
    intervals = {}
    for job_id in range(num_jobs):
        for task_id, (machine_id, duration) in enumerate(jobs_data[job_id]):
            # a job is consisted of multiple tasks

            unique_id = f'{job_id}-{machine_id}-{task_id}'
            start = model.NewIntVar(0, 10000, f's-{unique_id}')
            end = model.NewIntVar(0, 10000, f'e-{unique_id}')
            interval = model.NewIntervalVar(start, duration, end, f'interval-{unique_id}')

            # uniquely identified by job ID and machine ID
            # since a task can only be executed on a machine
            intervals[(job_id, machine_id)] = (start, end, interval)


    """Add constraints on the order"""
    for job_id in range(num_jobs):
        for task_id in range(1, num_machines):
            # (task number = machine number) in flow-shop

            prev_task_machine = jobs_data[job_id][task_id - 1][0]
            cur_task_machine = jobs_data[job_id][task_id][0]

            # start time of current task must be larger than the end time of previous task
            model.Add(intervals[(job_id, cur_task_machine)][0]
                      >= intervals[(job_id, prev_task_machine)][1])


    """Add constraints on the machine conflicts"""
    for machine_id in range(num_machines):
        machine_intervals = [
            intervals[(job_id, machine_id)][2]
            for job_id in range(num_jobs)
        ]
        model.AddNoOverlap(machine_intervals)


    """Setting Objects"""
    case_object = model.NewIntVar(0, 10000, 'makespan')
    model.AddMaxEquality(case_object, [
        intervals[(job_id, num_machines-1)][1]
        for job_id in range(num_jobs)
    ])
    model.Minimize(case_object)


    """Solving"""
    solver = cp_model.CpSolver()
    stat = solver.Solve(model)


    """Results"""
    result_schedule = []
    if stat == cp_model.OPTIMAL:
        # exists optimal solution
        print(f'Optimal Makespan: {solver.ObjectiveValue()}')
        for job_id in range(num_jobs):
            for machine_id in range(num_machines):
                start = solver.Value(intervals[(job_id, machine_id)][0])
                duration = jobs_data[job_id][machine_id][1]
                end = start + duration
                # print(f'Task-{machine_id} of Job-{job_id} is scheduled on Machine-{machine_id}: {start} ~ {end}')
                result_schedule.append((job_id, machine_id, start, end))
    else:
        # No optimal solution
        print('No solution found')

    return result_schedule


This may take a long time...

In [ ]:
for ins in instances:
    (n_jobs, n_machines), jobs_data = read_cases(ins)
    print(f"Problem: {ins} | Jobs: {n_jobs} | Machine: {n_machines}")

    start = time.time()
    result_schedule = solve_flowshop(n_jobs, n_machines, jobs_data)
    time_spent = time.time() - start

    machine_0_ops = [entry for entry in result_schedule if entry[1] == 0]
    machine_0_ops.sort(key=lambda x: x[2])
    sequence = [entry[0] for entry in machine_0_ops]
    print(f"Sequence: {sequence}")
    print(f"Time: {time_spent}\n")

    # plot_gantt_chart(result_schedule, n_jobs, n_machines, f'Result of case: {case_set_name}-{case_no}')

### Naïve Application on FunSearch

In [8]:
from evolved_func_test.utils import *

This is the evolved NEH function using the naive application on FunSearch:

In [9]:
def evolved_neh(processing_times: np.ndarray) -> list[int]:
    """
    An enhanced initial heuristic for the Permutation Flowshop Scheduling Problem (PFSP).

    This heuristic combines:
    - A weighted scoring for each job based on its total processing time and its maximum processing time.
      The weight parameter alpha balances these two criteria.
    - An iterative insertion procedure that builds an initial sequence.
    - A subsequent local search using pairwise swap improvements to further reduce the makespan.

    The resulting schedule (a list of job indices) is returned.
    """
    """
    An improved heuristic for the Permutation Flowshop Scheduling Problem (PFSP) that minimizes makespan
    by using a modified job ordering and insertion strategy.

    The heuristic performs the following steps:
    - Orders jobs based on their maximum processing time across all machines.
    - Builds an initial sequence using a modified greedy insertion strategy.
    - Applies a local search with pairwise swaps to optimize the sequence further.

    The resulting schedule (a list of job indices) is returned.
    """
    num_jobs, num_machines = processing_times.shape

    # Step 1: Order jobs based on their maximum processing time across all machines
    job_indices = np.arange(num_jobs)
    job_order = job_indices[np.argsort(-processing_times.max(axis=1))].tolist()

    # Step 2: Build an initial sequence using a modified greedy insertion strategy
    sequence = []
    for job in job_order:
        best_position = 0
        best_makespan = float('inf')

        # Try inserting the job in every possible position
        for pos in range(len(sequence) + 1):
            candidate_seq = sequence[:pos] + [job] + sequence[pos:]
            ms = calc_makespan(candidate_seq, processing_times)
            if ms < best_makespan:
                best_makespan = ms
                best_position = pos

        # Insert the job at the best position found
        sequence.insert(best_position, job)

    # Step 3: Local search: try pairwise swaps to further improve the sequence
    improvement = True
    while improvement:
        improvement = False
        current_makespan = calc_makespan(sequence, processing_times)

        for i in range(num_jobs - 1):
            for j in range(i + 1, num_jobs):
                new_seq = sequence.copy()
                new_seq[i], new_seq[j] = new_seq[j], new_seq[i]
                new_makespan = calc_makespan(new_seq, processing_times)

                if new_makespan < current_makespan:
                    sequence = new_seq
                    current_makespan = new_makespan
                    improvement = True
                    # Break out to restart the search after any improvement
                    break
            if improvement:
                break

    return sequence

Evaluation:

In [12]:
for ins in instances:
    directory = '/'.join(ins.split('/')[:-1])
    filename = ins.split('/')[-1]
    fs_data = load_datasets(directory)[filename]
    fs_data = np.array(fs_data)

    num_jobs, num_machines = fs_data.shape

    print(f"Problem: {ins} | Jobs: {num_jobs} | Machines: {num_machines}")
    
    start_time = time.time()
    schedule = evolved_neh(fs_data)
    time_spent = time.time() - start_time

    final_makespan = calc_makespan(schedule, fs_data)

    print(f"Sequence: {schedule}")
    print(f"Makespan: {final_makespan}")
    print(f"Time: {time_spent}\n")

    # plot_gantt_chart(schedule, fs_data)

    

Problem: data/carlier/carlier1.txt | Jobs: 11 | Machines: 5
Sequence: [7, 0, 4, 2, 10, 3, 5, 6, 8, 1, 9]
Makespan: 7038.0
Time: 0.01014399528503418

Problem: data/carlier/carlier2.txt | Jobs: 13 | Machines: 4
Sequence: [6, 10, 12, 2, 3, 4, 0, 1, 8, 7, 11, 9, 5]
Makespan: 7376.0
Time: 0.009727001190185547

Problem: data/heller/heller2.txt | Jobs: 20 | Machines: 10
Sequence: [0, 13, 12, 3, 8, 19, 7, 17, 16, 15, 2, 5, 11, 18, 14, 9, 1, 6, 10, 4]
Makespan: 139.0
Time: 0.0347440242767334

Problem: data/reeves/reeves10.txt | Jobs: 30 | Machines: 10
Sequence: [13, 12, 28, 5, 6, 4, 17, 21, 10, 2, 9, 23, 19, 1, 20, 8, 0, 14, 29, 3, 26, 24, 22, 7, 11, 16, 15, 18, 25, 27]
Makespan: 2132.0
Time: 0.17785286903381348

Problem: data/reeves/reeves15.txt | Jobs: 30 | Machines: 15
Sequence: [28, 14, 25, 3, 22, 5, 11, 24, 6, 27, 9, 13, 10, 23, 0, 29, 4, 19, 1, 15, 16, 21, 8, 2, 20, 26, 17, 12, 7, 18]
Makespan: 2391.0
Time: 0.336378812789917

Problem: data/reeves/reeves20.txt | Jobs: 75 | Machines: 20
Seq

## New Approaches

### Prompt Engineering

### FunSearch with Curriculum Learning

This is our key improvements that has made on the basic FunSearch Framework. The main idea of the new framework is applying **Curriculum Learning** in the evolving process of FunSearch.

Specifically, a single iteration of evolving is now turned into multiple iterations. In this framework, we call them 'Stages':
- The input instances are divided into various stages, starting from 'easy' instances all the way up to 'complicated' instances
    - Degrees of complication are defined manually
- At each stage, FunSearch is executed only with the instances belong to that stage, and gets the result
- If the result of current stage is higher than the baseline score, then it enters the next stage, namely a more complicated stage
    - Baseline function that provides baseline score is given by the raw function at each stage before the actual evolving
- If the result of current stage is lower than the baseline score, it keeps trying until it reaches the maximum number of attempts defined in advance, or gets a better score and escapes current stage
- The final output of this framework can either be the 'semi-evolved' or 'completely-evolved' function due to the maximum attempts limit

> See directory `implementation_cl` for the detailed implementation of this framework.


This is the evolved NEH function using our FunSearch-CL framework:

In [13]:
def compute_makespan(schedule: list[int], processing_times: np.ndarray) -> int:
    """
    Compute the makespan (total completion time) for a given job schedule in a PFSP.
    - schedule: list of job indices in the order they are processed.
    - processing_times: 2D numpy array of shape (num_jobs, num_machines) with processing times for each job on each machine.
    Returns the makespan (int) for the given order.
    """
    num_jobs = len(schedule)
    num_machines = processing_times.shape[1]
    if num_jobs == 0:
        return 0

    completion_times = np.zeros((num_jobs, num_machines), dtype=int)
    first_job = schedule[0]
    completion_times[0, 0] = processing_times[first_job, 0]
    for m in range(1, num_machines):
        completion_times[0, m] = completion_times[0, m - 1] + processing_times[first_job, m]

    for i in range(1, num_jobs):
        job = schedule[i]
        completion_times[i, 0] = completion_times[i - 1, 0] + processing_times[job, 0]
        for m in range(1, num_machines):
            completion_times[i, m] = max(completion_times[i, m - 1], completion_times[i - 1, m]) + processing_times[
                job, m]

    return int(completion_times[-1, -1])


def evolved_neh_cl(processing_times: np.ndarray) -> list[int]:
    import random
    num_jobs, num_machines = processing_times.shape

    def compute_priority_scores():
        scores = []
        weights = np.linspace(1.5, 0.5, num=num_machines)
        weighted_sums = processing_times @ weights
        for j in range(num_jobs):
            bottleneck = np.max(processing_times[j])
            score = 0.7 * weighted_sums[j] + 0.2 * processing_times[j].sum() + 0.1 * bottleneck
            scores.append((j, score))
        return sorted(scores, key=lambda x: -x[1])

    def dynamic_insertion(seq, job_id):
        best_seq = None
        best_makespan = float('inf')
        for i in range(len(seq) + 1):
            candidate = seq[:i] + [job_id] + seq[i:]
            ms = compute_makespan(candidate, processing_times)
            if ms < best_makespan:
                best_makespan = ms
                best_seq = candidate
        return best_seq

    def balance_machine_load(sequence):
        loads = np.zeros((num_machines,))
        for job in sequence:
            loads += processing_times[job]
        return np.std(loads)

    def tabu_local_search(init_seq, tabu_tenure=5, max_iter=100):
        current_seq = init_seq[:]
        best_seq = current_seq[:]
        best_makespan = compute_makespan(best_seq, processing_times)
        tabu_list = {}
        iteration = 0

        while iteration < max_iter:
            neighborhood = []
            for i in range(num_jobs):
                for j in range(i + 1, num_jobs):
                    if (i, j) in tabu_list and tabu_list[(i, j)] > iteration:
                        continue
                    temp_seq = current_seq[:]
                    temp_seq[i], temp_seq[j] = temp_seq[j], temp_seq[i]
                    ms = compute_makespan(temp_seq, processing_times)
                    load_dev = balance_machine_load(temp_seq)
                    score = ms + 0.01 * load_dev
                    neighborhood.append((score, ms, (i, j), temp_seq))

            if not neighborhood:
                break

            neighborhood.sort()
            score, ms, move, candidate_seq = neighborhood[0]
            current_seq = candidate_seq[:]
            tabu_list[move] = iteration + tabu_tenure

            if ms < best_makespan:
                best_makespan = ms
                best_seq = current_seq[:]

            iteration += 1

        return best_seq

    def adaptive_restart(base_seq, num_restarts=4):
        best_seq = base_seq[:]
        best_ms = compute_makespan(best_seq, processing_times)
        for r in range(num_restarts):
            shuffled = base_seq[:]
            random.shuffle(shuffled)
            evolved = tabu_local_search(shuffled, max_iter=30)
            ms = compute_makespan(evolved, processing_times)
            if ms < best_ms:
                best_ms = ms
                best_seq = evolved
        return best_seq

    scored_jobs = compute_priority_scores()
    ordered_jobs = [j for j, _ in scored_jobs]

    sequence = []
    for job in ordered_jobs:
        sequence = dynamic_insertion(sequence, job)

    sequence = tabu_local_search(sequence)
    sequence = adaptive_restart(sequence, num_restarts=5)

    return sequence

Get the results:

In [14]:
for ins in instances:
    directory = '/'.join(ins.split('/')[:-1])
    filename = ins.split('/')[-1]
    fs_data = load_datasets(directory)[filename]
    fs_data = np.array(fs_data)

    num_jobs, num_machines = fs_data.shape

    print(f"Problem: {ins} | Jobs: {num_jobs} | Machines: {num_machines}")
    
    start_time = time.time()
    schedule = evolved_neh_cl(fs_data)
    time_spent = time.time() - start_time

    final_makespan = calc_makespan(schedule, fs_data)

    print(f"Sequence: {schedule}")
    print(f"Makespan: {final_makespan}")
    print(f"Time: {time_spent}\n")

    # plot_gantt_chart(schedule, fs_data)

Problem: data/carlier/carlier1.txt | Jobs: 11 | Machines: 5
Sequence: [7, 0, 2, 4, 3, 10, 8, 6, 1, 9, 5]
Makespan: 7038.0
Time: 0.4140651226043701

Problem: data/carlier/carlier2.txt | Jobs: 13 | Machines: 4
Sequence: [6, 2, 3, 10, 12, 0, 1, 8, 7, 11, 4, 9, 5]
Makespan: 7166.0
Time: 0.5460460186004639

Problem: data/heller/heller2.txt | Jobs: 20 | Machines: 10
Sequence: [0, 1, 15, 19, 13, 18, 14, 3, 12, 8, 7, 11, 16, 2, 17, 5, 9, 10, 6, 4]
Makespan: 137.0
Time: 4.241786956787109

Problem: data/reeves/reeves10.txt | Jobs: 30 | Machines: 10
Sequence: [13, 28, 10, 2, 17, 1, 4, 6, 23, 22, 20, 9, 16, 7, 8, 0, 19, 3, 26, 14, 12, 24, 21, 29, 25, 5, 11, 15, 18, 27]
Makespan: 2120.0
Time: 14.38571810722351

Problem: data/reeves/reeves15.txt | Jobs: 30 | Machines: 15
Sequence: [28, 14, 3, 22, 5, 11, 6, 13, 23, 27, 9, 12, 0, 21, 15, 25, 7, 1, 29, 24, 26, 19, 2, 10, 17, 8, 20, 16, 4, 18]
Makespan: 2332.0
Time: 20.20167088508606

Problem: data/reeves/reeves20.txt | Jobs: 75 | Machines: 20
Sequence:

## Evaluations

Then, we will compare these different approaches using various metrics.

- Models:
    - Baseline: Base NEH algorithm, Google OR-Tools, Base Evolved Function
    - New Approaches: FunSearch Utilizing Prompt Engineering, FunSearch Incorporating Curriculum Learning

- Metrics: Final Makespan, Rate of Increase in Makespan, Execution Time, Proportion of Improved Makespan Across All Datasets.